## STEP 1— Imports Libar

In [9]:
import tensorflow as tf
print(tf.__version__)


2.20.0


In [10]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


## STEP 2 —  Dataset Path

In [11]:
data_dir = "../data/raw/plant_disease_images"


## STEP 3 — Load Images

In [12]:
IMG_SIZE = 128

images = []
labels = []
class_names = []

for folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder)
    
    if os.path.isdir(folder_path):
        class_names.append(folder)

for label, folder in enumerate(class_names):
    folder_path = os.path.join(data_dir, folder)
    
    for img_name in os.listdir(folder_path)[:300]:  # speed limit
        img_path = os.path.join(folder_path, img_name)
        
        img = cv2.imread(img_path)
        if img is None:
            continue
            
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        
        images.append(img)
        labels.append(label)

X = np.array(images) / 255.0
y = to_categorical(labels)

print("Total Classes:", len(class_names))
print("Dataset Shape:", X.shape)


Total Classes: 15
Dataset Shape: (4352, 128, 128, 3)


## STEP 4 — Train/Test Split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


## STEP 5 — CNN Model

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(class_names), activation='softmax')
])


c:\Users\f\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## STEP 6 — Compile

In [15]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


## STEP 7 — Train

In [17]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32
)


Epoch 1/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 189s 1s/step - accuracy: 0.8035 - loss: 0.5933 - val_accuracy: 0.7945 - val_loss: 0.6659
Epoch 2/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 121s 766ms/step - accuracy: 0.8262 - loss: 0.5224 - val_accuracy: 0.7899 - val_loss: 0.6054
Epoch 3/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 84s 768ms/step - accuracy: 0.8544 - loss: 0.4123 - val_accuracy: 0.8163 - val_loss: 0.5805
Epoch 4/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 78s 716ms/step - accuracy: 0.8793 - loss: 0.3596 - val_accuracy: 0.8106 - val_loss: 0.6506
Epoch 5/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 74s 680ms/step - accuracy: 0.8926 - loss: 0.3126 - val_accuracy: 0.7738 - val_loss: 0.7562
Epoch 6/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 74s 680ms/step - accuracy: 0.9032 - loss: 0.2836 - val_accuracy: 0.8129 - val_loss: 0.6377
Epoch 7/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 74s 678ms/step - accuracy: 0.9086 - loss: 0.2634 - val_accuracy: 0.8014 - val_loss: 0.7039
Epoch 8/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 75s 686ms/step - accuracy: 0.9308 - loss: 0.

## STEP 8 — Save Model + Class Names

In [18]:
model.save("../models/disease_model.h5")

import joblib
joblib.dump(class_names, "../models/disease_classes.pkl")

print("Disease model saved!")


Disease model saved!
